## Software-in-the-Loop (SiL) Simulation

tbd:

- Add documentation
- Add mock instances for `vs.PrometheusSignal` and `vs.WatttimeSignal` 

In [ ]:
from datetime import datetime
import vessim as vs

# Required for running Mosaic in Jupyter notebooks (fixes asyncio event loop conflicts)
import nest_asyncio
nest_asyncio.apply()

In [ ]:
environment = vs.Environment(sim_start=datetime.now(), step_size=10)

server = vs.Actor(name="gpu", signal=vs.PrometheusSignal(
    prometheus_url="http://localhost:30826/prometheus",
    query=f"sum(DCGM_FI_DEV_POWER_USAGE)",
    username="username",
    password="xxxxxxxxxx"
))
solar = vs.Actor(name="solar", signal=vs.Trace.load(
    dataset="solcast2022_global",
    column="Berlin",
    params={"scale": 200, "start_time": sim_start}
))
battery = vs.SimpleBattery(
    capacity=10 * i,
    initial_soc=0.6  # Start at 60% charge
)

microgrid = environment.add_microgrid(
    name=f"gpu_cluster",
    actors=[server, solar],
    storage=battery if battery.capacity > 0 else None,
    grid_signals={"mci_index": vs.WatttimeSignal(
        username="username",
        password="xxxxxxxxxx",
        location=(52.5200, 13.4050),
    )},
)

environment.add_controller(vs.Api(microgrids, export_prometheus=True))
environment.run(until=3600*24, rt_factor=1)